### **Import packages**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from delta.tables import *

### **Define paths**

In [0]:
silver_path = "abfss://silver@stfinancedev.dfs.core.windows.net/s_payments"
gold_path = "abfss://gold@stfinancedev.dfs.core.windows.net/fact_payments"

### **Create fact_payments delta lake table schema**

In [0]:
%sql
CREATE TABLE IF NOT EXISTS finance_cata.gold.fact_payments(
  Paymentkey BIGINT GENERATED ALWAYS AS IDENTITY(START WITH 1 INCREMENT BY 1),
  PaymentID STRING,
  CustomerKey BIGINT,
  Accountkey BIGINT,
  Branchkey BIGINT,
  Transaction_amt DECIMAL(18,2),
  PaymentDatekey BIGINT,
  PaymentType STRING,
  PaymentAmount STRING,
  Status STRING,
  FraudScore STRING,
  FraudFlag STRING,
  CreatedAt TIMESTAMP
)USING DELTA 
LOCATION 'abfss://gold@stfinancedev.dfs.core.windows.net/fact_payments'

### **Incremental load**

In [0]:
if DeltaTable.isDeltaTable(spark, gold_path):
    fact_df = DeltaTable.forPath(spark, gold_path).toDF()
    max_datekey = fact_df.agg(max(col("PaymentDatekey")).alias("max_datekey"))\
                        .first()["max_datekey"]
else:
    max_datekey = None

if max_datekey:
    inc_df = spark.read.format("delta").load(silver_path)\
             .withColumn("PaymentDateTime", date_format(col("PaymentDateTime"), "yyyyMMdd").cast("bigint"))\
             .filter(col("PaymentDateTime") > lit(max_datekey))
else:
    inc_df = spark.read.format("delta").load(silver_path)\
             .withColumn("PaymentDateTime", date_format(col("PaymentDateTime"), "yyyyMMdd").cast("bigint"))

In [0]:
#create temp view 
inc_df.createOrReplaceTempView("fact_payments")

In [0]:
%sql
select * from fact_payments

### **Create Fact_payments**

In [0]:
df = spark.sql("""
               SELECT 
                fp.PaymentID ,
                dc.CustomerKey ,
                da.Accountkey ,
                db.Branchkey ,
                fp.TransactionID ,
                fp.PaymentDateTime ,
                fp.PaymentType ,
                fp.PaymentAmount ,
                fp.Status ,
                fp.FraudScore ,
                fp.FraudFlag 
               FROM fact_payments AS fp
               INNER JOIN finance_cata.gold.dim_customers AS dc
               ON fp.CustomerID = dc.CustomerID
               INNER JOIN finance_cata.gold.dim_accounts AS da
               ON fp.AccountID = da.AccountID
               LEFT JOIN finance_cata.gold.dim_branches AS db
               ON da.BranchID = db.BranchID
               """)

In [0]:
df.display()

### **Add stg columns**

In [0]:
df_ready = df.withColumn("TransactionID", col("TransactionID").cast("decimal(18,2)"))\
              .withColumnRenamed("TransactionID","Transaction_amt")\
              .withColumnRenamed("PaymentDateTime", "PaymentDateKey")\
              .withColumn("CreatedAt", current_timestamp())

In [0]:
df_ready.display()

### **Write in gold layer**

In [0]:
df_ready.write.format("delta").mode("append").save(gold_path)

In [0]:
%sql
select  * from finance_cata.gold.fact_payments